In [118]:
import re 
import string
import timestring
from sklearn.linear_model import LinearRegression, LogisticRegression
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import cross_val_score, KFold
import glob, os
import numpy as np
from sklearn import svm
from collections import defaultdict
from scipy.sparse import csr_matrix

In [119]:
vectorizer1 = TfidfVectorizer(decode_error='ignore', ngram_range=(1, 2), max_df=1., min_df=2,
                             use_idf=True, tokenizer=tokenize,binary=False, norm='l2')
vectorizer2 = TfidfVectorizer(decode_error='ignore', ngram_range=(1, 2), max_df=1., min_df=2,
                             use_idf=True, tokenizer=tokenize, binary=False, norm='l2')
#X1 = vectorizer1.fit_transform(t['text'] for t in positive_sample)
#X2 = vectorizer2.fit_transform(t['text'] for t in negative_sample)
#print('Vectorized %d tweets. Found %d terms.' % (X1.shape[0], X1.shape[1]))
#features = np.array(vectorizer1.get_feature_names())

In [229]:
#DIR = 'C:/Users/Elaine/Desktop/TCC/all timeline/'
import os, io, json, codecs

os.chdir(DIR)

positive_sample=[]
negative_sample = []
users = defaultdict(tuple)

for file in glob.glob("*.txt"):
    print 'file ', file, '\n'
    with io.open(DIR+file,'r',encoding="utf-8") as f:#, codecs.open("CleuzaBapti - Copy.txt", 'w+', encoding='utf-8') as fi:
        lines = f.readlines()
        i=0
        for i in xrange(len(lines)):
            tweet= json.loads(lines[i]) 

            if 'foraDILMA' in tweet['text'] or 'foradilma' in tweet['text'] or 'Fora Dilma' in tweet['text'] or 'foraDilma' in tweet['text'] or 'FORADILMA' in tweet['text'] or 'fora Dilma' in tweet['text'] or 'FORA DILMA' in tweet['text'] or 'ForaDilma' in tweet['text'] or'foraPT' in tweet['text'] or 'FORAPT' in tweet['text'] or 'fora pt' in tweet['text']or 'FORA PT' in tweet['text'] or 'ForaPT' in tweet['text'] or'vempraRua' in tweet['text'] or 'vemprarua' in tweet['text'] or 'vem pra rua' in tweet['text']or 'VEMPRARUA' in tweet['text'] or 'VemPraRua' in tweet['text']:
                j=i-1
                while j>1:
                    #print j
                    tweet1= json.loads(lines[j])    
                    #print tweet['time_tweet']
                    positive_sample.append(tweet1)
                    j=j-1
                    #if j == 1:
                    #    print 'start of the file'
                    #    break
                negative_sample = positive_sample[10:]

                #positive = ''
                #negative = ''
                #for p in positive_sample:
                #    positive = positive +" ".join(p)
                    
                #for n in negative_sample:
                #    negative = negative +" ".join(n)
                    
                X1 = vectorizer1.fit_transform(p['text'] for p in positive_sample)
                X2 = vectorizer2.fit_transform(n['text'] for n in negative_sample)
                
                train = (X1,X2)
                y = (1,0)

                users[file] = train,y

                break
            X1 = []
            X2 = []
        


file  AntoniCorreaa.txt.txt 

file  antonionetoAC.txt.txt 

file  Anyssima.txt.txt 

file  ariadnimariano.txt.txt 

file  Aristotelesbh.txt.txt 



KeyboardInterrupt: 

In [230]:
print users

defaultdict(<type 'tuple'>, {'antonionetoAC.txt.txt': ((<863x2252 sparse matrix of type '<type 'numpy.float64'>'
	with 12307 stored elements in Compressed Sparse Row format>, <853x2237 sparse matrix of type '<type 'numpy.float64'>'
	with 12212 stored elements in Compressed Sparse Row format>), (1, 0)), 'AntoniCorreaa.txt.txt': ((<33x63 sparse matrix of type '<type 'numpy.float64'>'
	with 202 stored elements in Compressed Sparse Row format>, <23x48 sparse matrix of type '<type 'numpy.float64'>'
	with 139 stored elements in Compressed Sparse Row format>), (1, 0)), 'ariadnimariano.txt.txt': ((<6559x11736 sparse matrix of type '<type 'numpy.float64'>'
	with 89121 stored elements in Compressed Sparse Row format>, <6549x11722 sparse matrix of type '<type 'numpy.float64'>'
	with 89000 stored elements in Compressed Sparse Row format>), (1, 0)), 'Anyssima.txt.txt': ((<3722x8397 sparse matrix of type '<type 'numpy.float64'>'
	with 58873 stored elements in Compressed Sparse Row format>, <3712x838

In [232]:
t = [zip(v[0],v[1]) for k,v in users.items()]
Y=[]
XX=[]
for b in t:
    Y.append(b[0][1])
    Y.append(b[1][1])
    XX.append(b[0][0])
    XX.append(b[1][0])
    
print Y
print XX




[1, 0, 1, 0, 1, 0, 1, 0]
[<863x2252 sparse matrix of type '<type 'numpy.float64'>'
	with 12307 stored elements in Compressed Sparse Row format>, <853x2237 sparse matrix of type '<type 'numpy.float64'>'
	with 12212 stored elements in Compressed Sparse Row format>, <33x63 sparse matrix of type '<type 'numpy.float64'>'
	with 202 stored elements in Compressed Sparse Row format>, <23x48 sparse matrix of type '<type 'numpy.float64'>'
	with 139 stored elements in Compressed Sparse Row format>, <6559x11736 sparse matrix of type '<type 'numpy.float64'>'
	with 89121 stored elements in Compressed Sparse Row format>, <6549x11722 sparse matrix of type '<type 'numpy.float64'>'
	with 89000 stored elements in Compressed Sparse Row format>, <3722x8397 sparse matrix of type '<type 'numpy.float64'>'
	with 58873 stored elements in Compressed Sparse Row format>, <3712x8382 sparse matrix of type '<type 'numpy.float64'>'
	with 58753 stored elements in Compressed Sparse Row format>]


In [236]:
print('Vectorized %d tweets. Found %d terms.' % (XX[1].shape[0], XX[1].shape[1]))
print('Vectorized %d tweets. Found %d terms.' % (X2.shape[0], X2.shape[1]))

Vectorized 853 tweets. Found 2237 terms.


AttributeError: 'list' object has no attribute 'shape'

In [235]:
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression

def do_cv(X, y, nfolds=3):
    cv = KFold(len(y), nfolds)
    return np.mean(cross_val_score(LogisticRegression(), X, y, cv=cv))
  
print do_cv(XX, Y)

ValueError: setting an array element with a sequence.

In [56]:
with codecs.open("C:/Users/Elaine/"+'file_test', 'w+', encoding='utf-8') as fi:
       for f in features:
            fi.write(f)

In [68]:
def tokenize(text):
    punc_re = '[' + '\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\_\\{\\|\\}' + ']'
    #print punc_re
    text = text.lower()
    text = re.sub('#(\S+)', r'HASHTAG_\1', text)
    text = re.sub('@(\S+)', r'MENTION_\1', text)
    text = re.sub('http\S+', 'THIS_IS_A_URL', text)
    text = re.sub(r'(.)\1\1\1+', r'\1', text)
    text = re.sub(r'[0-9]', '9', text)
    toks = []
    for tok in text.split():
        tok = re.sub(r'^(' + punc_re + '+)', r'\1 ', tok)
        #print '1',tok
        tok = re.sub(r'(' + punc_re + '+)$', r' \1', tok)
        #print '2',tok

        for subtok in tok.split():
            #print '3',subtok
            if re.search('\w', subtok):
                toks.append(subtok)
    return toks